## Assignment-07 First Step of using machine learning and models.

![](https://github.com/Computing-Intelligence/jupyters_and_slides/raw/master/entrance-test/images/kkb-ai.png)

### Step1: 数据分析

请在课程的GitHub中下载数据集，然后使用pandas进行读取。

### Step2: 数据预处理

将pandas中的数据，依据是否是新华社的文章，请改变成新的数据dataframe: <content, y>, 其中，content是文章内容，y是0或者1. 你可能要使用到pandas的dataframe操作。https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html

### Step3: 使用tfidf进行文本向量化

参考 https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html 对

对文本进行向量化

### Step4: 参考scikit-learning的方法，构建你的第一个机器学习模型

+ 按照课程讲解的内容，把数据集分割为 traning_data, validation_data, test_data. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

+ 参照scikit learning的示例，从构建你的第一个KNN机器学习模型。

### Step5: 在traning_data, validation_data, test_data 上观察其相关metric: recall, precision, f1等， 并解释其含义. 

### Step6: 使用 test_data 对其进行新数据的判断, 你有没

### Step7: 调整不同的参数，观察变化

### Step8: 不断改变参数，直到性能达到“某个”点。问：“某个”怎么定义？

### Step9: 找出所以预测为 1， 但是实际为 0 的文章。 作为抄袭的候选者。

作业代码实现:

In [3]:
import pandas as pd
import numpy as np
import jieba
from typing import List,Dict
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score


class MyKnn_Verify_Copy_News:
    def __init__(self,
                 data_path:str='/home/abc/下载/NLP_assessment/lesson7+assessment7/sqlResult_1558435.csv',
                 model_type='nb'):
        self.__data_path=data_path
        self.__model_type=model_type

    # def __is_chinese(self,uchar:str):
    #     # """判断一个unicode是否是汉字"""
    #     if u'\u4e00' <= uchar <= u'\u9fff':
    #         return True

    # def __clean_line(self,s:str)->str:
    #     """
    #     :param s: 清洗爬取的中文语料格式
    #     :return:
    #     """
    #     import re
    #     from string import digits, punctuation
    #     rule = re.compile(u'[^a-zA-Z.,;《》？！“”‘’@#￥%…&×（）——+【】{};；●，。&～、|\s:：' + digits + punctuation + '\u4e00-\u9fa5]+')
    #     s = re.sub(rule, '', s)
    #     s = re.sub('[、]+', '，', s)
    #     s = re.sub('\'', '', s)
    #     s = re.sub('[#]+', '，', s)
    #     s = re.sub('[?]+', '？', s)
    #     s = re.sub('[;]+', '，', s)
    #     s = re.sub('[,]+', '，', s)
    #     s = re.sub('[!]+', '！', s)
    #     s = re.sub('[.]+', '.', s)
    #     s = re.sub('[，]+', '，', s)
    #     s = re.sub('[。]+', '。', s)
    #     s = s.strip().lower()
    #     return s

#    def __cut(self,string):
#        return ' '.join(jieba.cut(string))

    def __clean_data(self)->pd.DataFrame:
        print('开始下载和清洗数据')
        self.__data = pd.read_csv(self.__data_path,encoding='gb18030')
        self.__data=self.__data.fillna('')
        self.__data['content']=self.__data['content'].apply(lambda x:' '.join(jieba.cut(x)))
        self.__data['news_labels']=np.where(self.__data['source'].str.contains('新华社'), 1, 0)
        ratio=len(self.__data[self.__data['news_labels']==1])/ \
               len(self.__data)
        print('取得的新华社新闻占比为%0.5f%s' %
              (ratio*100,'%'))


    def get_original_labels(self)->pd.Series:
        return self.__data['news_labels']

    # 获得角度矩阵和拆分数据集
    def __TfidfVectorizer(self)->List[str]:
        print('我们有数据%d行%d列'%(self.__data.shape[0],self.__data.shape[1]))
        train_size=int(input('输入训练集大小(最大推荐为50000): '))
        if not train_size:
            raise ('训练集大小不能为空')
        max_f=int(input('输入最大特征数量(推荐最大为3000): '))
        if not max_f:
            raise('特征最大数量不能为空')
        vectorizer = TfidfVectorizer(max_features=max_f)
        total_data=self.__data['content'].values[:train_size]
        total_labels=self.__data['news_labels'].values[:train_size]
        total_data=vectorizer.fit_transform(total_data)
        print('我们得到了以下大小的词向量角度矩阵:')
        print(total_data.shape)
        test_size=float(input('输入想要拆分数据集的百分比:'))
        x_train,x_test,y_train,y_test=train_test_split(total_data,total_labels,
                                                       test_size=test_size,
                                                       random_state=0)
        return x_train,x_test,y_train,y_test,total_data,total_labels

    # nb+dt
    def __training_model(self,model_type,x_train:List[float],y_train:List[int],weight:List[int]=None,
                         alpha:float=0.01,binarize:float=0.0,k:int=None,
                         c:float=None,gamma:float=None,n_estimators=None,
                         max_depth=None,min_samples_split=None,max_features=None):
        if model_type=='nb':
            if len(weight)==0 and not isinstance(weight,list):
                model=GaussianNB()
                model.fit(x_train.toarray(),y_train)
            else:
                model=GaussianNB(priors=weight)
                model.fit(x_train.toarray(),y_train)
        elif model_type=='bnb':
            # 不设定过大特征个数
            model=BernoulliNB(alpha=alpha,binarize=binarize,
                              fit_prior=True,class_prior=None)
            model.fit(x_train.toarray(),y_train)
        elif model_type=='knn':
            model=KNeighborsClassifier(n_neighbors=k)
            model.fit(x_train.toarray(),y_train)
        elif model_type=='svm':
            model=SVC(C=c,kernel='rbf',gamma=gamma,max_iter=2000)
            model.fit(x_train.toarray(),y_train)
        elif model_type=='logistic':
            model=LogisticRegression(solver='sag',n_jobs=-1)
            model.fit(x_train.toarray(),y_train)
        elif model_type=='forest':
            model=RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth,
                                        min_samples_split=min_samples_split,max_features=max_features)
            model.fit(x_train.toarray(),y_train)
        else:
            raise('错误的模型')
        return model

    def __predict(self,model,x_,y_):
        prediction=model.predict(x_.toarray())
        accurate=model.score(x_.toarray(),y_)
        print('得分是%0.5f' % accurate)
        return prediction


    # 混交矩阵和三种指标
    def __performance_indicator(self,model,
                                x_train:List[float],
                                y_train:List[int],
                                sample=1000):
        precision=None
        recall=None
        roc_aoc=None
        confusion_max=None
        random_set = np.random.choice(np.arange(len(y_train)), sample)
        sub_x = x_train[random_set]
        sub_y = y_train[random_set]
        y_hat = model.predict(sub_x.toarray())
        model_score=model.score(sub_x.toarray(),sub_y)
        print('准确率: {}'.format(precision_score(sub_y, y_hat)))
        precision=precision_score(sub_y, y_hat)
        print('召回率: {}'.format(recall_score(sub_y, y_hat)))
        print('ROC_AOC曲线: {}'.format(roc_auc_score(sub_y, y_hat)))
        print('混肴矩阵: \n{}'.format(confusion_matrix(sub_y, y_hat, labels=[0, 1])))
        print('模型打分: {}'.format(model_score))
        return precision

    def verify_atricle(self,prediction_labels:List[int])->float:
        j=1
        for i in range(len(self.__data)):
            if (prediction_labels[i]==1) and (self.__data.iloc[i]['news_labels']==0) and\
                    self.__data.iloc[i]['content']:
                print('来源:{0},内容是{1},被判断是抄袭新华社的消息' .format(
                    self.__data.iloc[i]['source'],
                    self.__data.iloc[i]['content']
                ))
                j+=1
            else:
                continue
        print('一共%d条新闻被认定是抄袭新华社'%j)
        ratio=j/len(self.__data)
        print('判别出的抄袭新闻占总新闻的%0.2f%s' % (ratio*100,'%'))
        return ratio*100

    def main(self,sample_num=1000):
        self.__clean_data()
        print('数据清洗完毕')
        # 没有办法数据归一化,既STANDARD_SCALER(x_train.toarray())
        x_train,x_test,y_train,\
        y_test ,total_data,total_labels=self.__TfidfVectorizer()
        
        if self.__model_type=='nb':
            print('开始训练朴素贝叶斯模型')
            # 用普通朴素贝叶斯模型形成的结果
            training_score={}
            testing_score={}
            models=[]
            i=1
            for m,n in zip(np.linspace(0.2,1.0,5),np.linspace(0.8,0,5)):
                fitted_model=self.__training_model('nb',x_train,y_train,weight=[m,n])
                models.append(fitted_model)
                print('模型训练完毕')
                print('-'*40)
                print('进行模型准确率->ROC_AOC曲线表现打分')
                # 通过选取不同大小的随机样本集进行测试
                print('第%d批次训练集准确率->ROC_AOC曲线打分'%i)
                training_score[i]=self.__performance_indicator(fitted_model,x_train,y_train,sample_num)
                print('-'*40)
                print('第%d批次测试集准确率->ROC_AOC曲线打分'%i)
                testing_score[i]=self.__performance_indicator(fitted_model,x_test,y_test,sample_num)
                i += 1
                print('-'*40)
            print('模型打分结束')
            print('-'*40)
            print('选择模型打分最高的模型')
            # 以测试集合的最高准确率为选择标准
            highest_score=max(testing_score,key=lambda x:testing_score[x])
            best_fitted_model=models[highest_score-1]
            print('准确率最高的是第%d个模型,准确率是%0.2f'% (highest_score-1,testing_score[highest_score]))
            print('-' * 40)
            print('对固定训练集和测试集进行预测开始')
            print('固定训练集预测和打分')
            train_predictions=self.__predict(best_fitted_model,x_train,y_train)
            print('固定测试集预测和打分')
            test_predictions=self.__predict(best_fitted_model,x_test,y_test)
            print('-'*40)
            print('对全体文章进行预测')
            total_predcitons=self.__predict(best_fitted_model,total_data,total_labels)
            print('普通朴素贝叶斯模型结束')
            return training_score, testing_score,total_predcitons


        elif self.__model_type=='bnb':
            print('-'*40)
            print('伯努利贝叶斯模型开始')
            print('开始训练模型')
            models=[]
            i=1
            training_score = {}
            testing_score = {}
            for part_alpha,part_binarize in zip(np.linspace(0.1,1.0,10),np.linspace(1,10,10)):
                fitted_model = self.__training_model('bnb', x_train, y_train,
                                                     alpha=part_alpha,
                                                     binarize=part_binarize)

                print('alpha:%0.2f,binarize:%d的模型训练完毕' % (part_alpha,part_binarize))
                models.append(fitted_model)
                print('-' * 40)
                print('进行模型准确率->ROC_AOC曲线表现打分')
                # 通过选取不同大小的随机样本集进行测试
                print('第%d批次训练集准确率->ROC_AOC曲线打分' % i)
                training_score[i]=self.__performance_indicator(fitted_model, x_train, y_train, sample_num)
                print('-' * 40)
                print('第%d批次测试集准确率->ROC_AOC曲线打分' % i)
                testing_score[i]=self.__performance_indicator(fitted_model, x_test, y_test, sample_num)
                i += 1
                print('-' * 40)
            print('模型打分结束')
            precision_list=list(training_score.values())
            print(precision_list)
            plt.plot(np.linspace(0.1,1.0,10),precision_list)
            plt.plot(np.linspace(1,10,10),precision_list)
            plt.xlim([0,10])
            plt.xlabel('alpha_binarize参数')
            plt.xlabel('trainning_score')
            plt.legend('alpha-score','binarize-score')
            plt.title('bnb_alpha_binarize_score')
            plt.show()
            print('-' * 40)
            print('选择模型打分最高的模型')
            # 以准确率为选择标准
            highest_score=max(testing_score,key=lambda x:testing_score[x])
            best_fitted_model=models[highest_score-1]
            print('准确率最高的是第%d个模型,准确率是%0.2f'% (highest_score-1,testing_score[highest_score]))
            print('-' * 40)
            print('对训练集和测试集进行预测开始')
            print('训练集预测和打分')
            train_predictions = self.__predict(best_fitted_model, x_train, y_train)
            print('测试集预测和打分')
            test_predictions = self.__predict(best_fitted_model, x_test, y_test)
            print('对全体文章进行预测')
            total_predcitons = self.__predict(best_fitted_model, total_data, total_labels)
            return training_score, testing_score,total_predcitons

        elif self.__model_type=='knn':
            print('-'*40)
            print('K邻近模型开始')
            print('开始训练模型')
            # 调参数k,根据准确性选择最佳k和模型
            training_score =None
            testing_score = None
            fitted_model = self.__training_model('knn', x_train, y_train,k=2)
            print('模型训练完毕')
            print('进行模型准确率->ROC_AOC曲线表现打分')
            # 通过选取不同大小的随机样本集进行测试
            print('训练集准确率->ROC_AOC曲线打分')
            training_score=self.__performance_indicator(fitted_model, x_train, y_train, sample_num)
            print('-' * 40)
            print('测试集准确率->ROC_AOC曲线打分')
            testing_score=self.__performance_indicator(fitted_model, x_test, y_test, sample_num)
            print('模型打分结束')
            print('-' * 40)
            print('对训练集和测试集进行预测开始')
            print('训练集预测和打分')
            train_predictions = self.__predict(fitted_model, x_train, y_train)
            print('测试集预测和打分')
            test_predictions = self.__predict(fitted_model, x_test, y_test)
            print('对全体文章进行预测')
            total_predcitons = self.__predict(fitted_model,total_data,total_labels)
            return training_score, testing_score,total_predcitons

        elif self.__model_type=='svm':
            print('-' * 40)
            print('svm模型开始')
            print('开始训练模型')
            # 调参数k,根据准确性选择最佳k和模型
            best_c=[]
            best_g=[]
            training_score = {}
            testing_score = {}
            models = []
            i = 1
            for c,g in zip(np.linspace(0.1,1.0,5),np.linspace(0.2,1.0,5)):
                fitted_model = self.__training_model('svm', x_train, y_train,
                                                     c=c,gamma=g)
                print('c=%0.5f和gamma=%0.5f的模型训练完毕' % (c,g))
                models.append(fitted_model)
                best_c.append(c)
                best_g.append(g)
                print('进行模型准确率->ROC_AOC曲线表现打分')
                # 通过选取不同大小的随机样本集进行测试
                print('第%d批次训练集准确率->ROC_AOC曲线打分' % i)
                training_score[i] = self.__performance_indicator(fitted_model, x_train, y_train, sample_num)
                print('-' * 40)
                print('第%d批次测试集准确率->ROC_AOC曲线打分' % i)
                testing_score[i] = self.__performance_indicator(fitted_model, x_test, y_test, sample_num)
                i += 1
                print('-' * 40)
            print('模型打分结束')
            print('-' * 40)
            print('选择模型打分最高的模型')
            highest_score = max(testing_score, key=lambda x: testing_score[x])
            best_fitted_model = models[highest_score - 1]
            print('准确率最高的是第%d个模型,c=%0.2f,gamma=%0.2f,准确率是%0.2f' %
                  (highest_score,best_c[highest_score-1],
                   best_g[highest_score-1],testing_score[highest_score]))
            print('-' * 40)
            print('对训练集和测试集进行预测开始')
            print('训练集预测和打分')
            train_predictions = self.__predict(best_fitted_model, x_train, y_train)
            print('测试集预测和打分')
            test_predictions = self.__predict(best_fitted_model, x_test, y_test)
            print('对全体文章进行预测')
            total_predcitons = self.__predict(best_fitted_model,
                                              total_data,
                                              total_labels)
            return training_score, testing_score, total_predcitons
        
        elif self.__model_type=='logistic':
            print('-' * 40)
            print('logistic模型开始')
            training_score=None
            testing_score=None
            print('开始训练模型')
            fitted_model = self.__training_model('logistic', x_train, y_train)
            print('模型训练完毕')
            print('进行模型准确率->ROC_AOC曲线表现打分')
            # 通过选取不同大小的随机样本集进行测试
            print('训练集准确率->ROC_AOC曲线打分')
            training_score = self.__performance_indicator(fitted_model, x_train, y_train, sample_num)
            print('-' * 40)
            print('测试集准确率->ROC_AOC曲线打分')
            testing_score = self.__performance_indicator(fitted_model, x_test, y_test, sample_num)
            print('-' * 40)
            print('模型打分结束')
            print('-' * 40)
            print('对训练集和测试集进行预测开始')
            print('训练集预测和打分')
            train_predictions = self.__predict(fitted_model, x_train, y_train)
            print('测试集预测和打分')
            test_predictions = self.__predict(fitted_model, x_test, y_test)
            print('对全体文章进行预测')
            total_predcitons = self.__predict(fitted_model,
                                              total_data,
                                              total_labels)
            return training_score, testing_score, total_predcitons    
        
        
        elif self.__model_type=='forest':
            print('-' * 40)
            print('随机森林模型开始')
            print('开始训练模型')
            # 调参数k,根据准确性选择最佳k和模型
            best_iter=[]
            best_depth=[]
            best_samples=[]
            best_features=[]
            training_score = {}
            testing_score = {}
            models = []
            i = 1
            for iters,depth,samples,features in zip(range(10,100,10),range(3,13,1),range(100,200,10),
                                                    range(3,13,1)):
                fitted_model = self.__training_model('forest',x_train,y_train,n_estimators=iters,
                                                     max_depth=depth,min_samples_split=samples,
                                                     max_features=features)
                print('iter=%d,depth=%d,samples=%d和features=%d的模型训练完毕' %
                     (iters,depth,samples,features))
                models.append(fitted_model)
                best_iter.append(iters)
                best_depth.append(depth)
                best_samples.append(samples)
                best_features.append(features)
                print('进行模型准确率->ROC_AOC曲线表现打分')
                # 通过选取不同大小的随机样本集进行测试
                print('第%d批次训练集准确率->ROC_AOC曲线打分' % i)
                training_score[i] = self.__performance_indicator(fitted_model, x_train, y_train, sample_num)
                print('-' * 40)
                print('第%d批次测试集准确率->ROC_AOC曲线打分' % i)
                testing_score[i] = self.__performance_indicator(fitted_model, x_test, y_test, sample_num)
                i += 1
                print('-' * 40)
            print('模型打分结束')
            print('-' * 40)
            print('选择模型打分最高的模型')
            highest_score = max(testing_score, key=lambda x: testing_score[x])
            best_fitted_model = models[highest_score - 1]
            print('准确率最高的是第%d个模型,iter=%d,depth=%d,samples=%d和features=%d,准确率是%0.2f' %
                  (highest_score,best_iter[highest_score-1], best_depth[highest_score-1],
                   best_samples[highest_score-1],best_features[highest_score-1],
                   testing_score[highest_score]))
            print('-' * 40)
            print('对训练集和测试集进行预测开始')
            print('训练集预测和打分')
            train_predictions = self.__predict(best_fitted_model, x_train, y_train)
            print('测试集预测和打分')
            test_predictions = self.__predict(best_fitted_model, x_test, y_test)
            print('对全体文章进行预测')
            total_predcitons = self.__predict(best_fitted_model,
                                              total_data,
                                              total_labels)
            return training_score, testing_score, total_predcitons

        else:
            raise('错误的模型输入')






if __name__=='__main__':
    # print('朴素贝叶斯开始')
    # # 朴素贝叶斯模型,效果很差
    # bn_model=MyKnn_Verify_Copy_News(model_type='nb')
    # training_score, testing_score,total_predcitons=bn_model.main()
    # # 获取处理过后的全数据
    # original_labels=bn_model.get_original_labels()
    # # 输出被判定为抄袭的文章
    # bn_ratio=bn_model.verify_atricle(total_predcitons)
    # print('朴素贝叶斯结束')
    # print('-'*40)
    # print('伯努利模型开始')
    # # 伯努利模型 效果很明显
    # bnb_model=MyKnn_Verify_Copy_News(model_type='bnb')
    # training_score, testing_score, total_predcitons = bnb_model.main()
    # original_labels = bnb_model.get_original_labels()
    # bnb_ratio=bnb_model.verify_atricle(total_predcitons)
    # print('伯努利模型结束')

    # k邻近,时间过久
#     print('k邻近模型开始')
#     knn_model=MyKnn_Verify_Copy_News(model_type='knn')
#     training_score, testing_score, total_predcitons = knn_model.main()
#     original_labels = knn_model.get_original_labels()
#     knn_model=knn_model.verify_atricle(total_predcitons)
#     print('k邻近模型结束')


#     # svm->由于时间过长,将max_iter=800,但效果很差,无法对全体文章进行有效识别
#     print('svm模型开始')
#     svm_model=MyKnn_Verify_Copy_News(model_type='svm')
#     training_score, testing_score, total_predcitons = svm_model.main()
#     original_labels = svm_model.get_original_labels()
#     knn_model=svm_model.verify_atricle(total_predcitons)
#     print('svm模型结束')

    # logistic模型
#     print('logistic模型开始')
#     logistic_model=MyKnn_Verify_Copy_News(model_type='logistic')
#     training_score, testing_score, total_predcitons = logistic_model.main()
#     original_labels = logistic_model.get_original_labels()
#     knn_model=logistic_model.verify_atricle(total_predcitons)
#     print('logistic模型结束')

    print('随机森林模型开始')
    forest_model=MyKnn_Verify_Copy_News(model_type='forest')
    training_score, testing_score, total_predcitons = forest_model.main()
    original_labels = forest_model.get_original_labels()
    knn_model=forest_model.verify_atricle(total_predcitons)
    print('随机森林模型结束')


随机森林模型开始
开始下载和清洗数据
取得的新华社新闻占比为87.79503%
数据清洗完毕
我们有数据89611行8列
输入训练集大小(最大推荐为50000): 89611
输入最大特征数量(推荐最大为3000): 3000
我们得到了以下大小的词向量角度矩阵:
(89611, 3000)
输入想要拆分数据集的百分比:0.25
----------------------------------------
随机森林模型开始
开始训练模型
iter=10,depth=3,samples=100和features=3的模型训练完毕
进行模型准确率->ROC_AOC曲线表现打分
第1批次训练集准确率->ROC_AOC曲线打分
准确率: 0.877
召回率: 1.0
ROC_AOC曲线: 0.5
混肴矩阵: 
[[  0 123]
 [  0 877]]
模型打分: 0.877
----------------------------------------
第1批次测试集准确率->ROC_AOC曲线打分
准确率: 0.875
召回率: 1.0
ROC_AOC曲线: 0.5
混肴矩阵: 
[[  0 125]
 [  0 875]]
模型打分: 0.875
----------------------------------------
iter=20,depth=4,samples=110和features=4的模型训练完毕
进行模型准确率->ROC_AOC曲线表现打分
第2批次训练集准确率->ROC_AOC曲线打分
准确率: 0.894
召回率: 1.0
ROC_AOC曲线: 0.5
混肴矩阵: 
[[  0 106]
 [  0 894]]
模型打分: 0.894
----------------------------------------
第2批次测试集准确率->ROC_AOC曲线打分
准确率: 0.879
召回率: 1.0
ROC_AOC曲线: 0.5
混肴矩阵: 
[[  0 121]
 [  0 879]]
模型打分: 0.879
----------------------------------------
iter=30,depth=5,samples=120和features=5的模型训练完毕
进行模型准确率->ROC_AOC曲线表现打分
第3

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




来源:参考消息网,内容是参考消息 网 6 月 19 日 报道   英媒称 ， 一名 俄罗斯 儿童 在 遍布 黑熊 和 狼群 的 森林 里 迷路 并且 独自 生存 了 五天 ， 最终 奇迹般地 生还 ， 被 人 找到 。 
 据 英国 《 太阳报 》 6 月 15 日 报道 ， 四岁 的 季马 · 佩 斯科夫 在 被 找到 时 已经 精疲力尽 ， 身上 到处 都 是 被 蚊子 咬 的 痕迹 。 他 的 父母 说 ， 他 受到 了 严重 惊吓 ， 无法 说话 。 他 是 在 随 家人 一起 在 乌拉尔山 的 一个 水库 附近 野营 时 走失 的 。 
 救援 人员 说 ， 这个 小家伙 靠着 吃 草 和 喝水 坑里 的 水 顽强 地活 了 下来 。 尽管如此 ， 这个 被 发现 时 蜷缩 在 一棵树 下 的 孩子 仍然 出现 了 脱水 和 肺炎 的 症状 。 
 他 的 医生 说 ， 他 的 病情 “ 严重 但是 稳定 ” 。 
 在 季马 被 发现 之前 ， 警方 甚至 已经 开始 怀疑 他 的 父亲 在 森林 里 把 他 杀害 并 埋葬 了 。 
 为了 寻找 季马 ， 当地 有 500 名 志愿者 参与 了 搜救 行动 ， 他 所 失踪 的 这篇 森林 到处 都 是 黑熊 和 狼群 。 一名 救援 人员 说 ， 他 能活 下来 简直 就是 奇迹 。 （ 编译 / 武林 ） 
,被判断是抄袭新华社的消息
来源:广州日报第FSA15版,内容是今明 雨弱   气温 走高 
 　 　 广州日报 讯   （ 全 媒体 记者 李传智 ） 虽然 湿哒 哒 的 雨水 天气 让 不少 街坊 出行 受到 影响 ， 但 凉爽 的 天气 却 让 人 留恋 。 气象部门 预测 ， 今明两天 ， 佛山 降雨 逐渐 减弱 ， 气温 将 有所 回升 。 市民 告别 雨水 的 同时 ， 还 需注意 补水 防暑 。 
 　 　 全天候 
 　 　 今日 天气 
 　 　 多云间阴 天 ， 有 雷阵雨 ， 25 ～ 32 ℃ ； 偏南风 2 级 ， 相对湿度 75% ~ 95% 
 　 　 明日 天气 
 　 　 多云 ， 局部 有 阵雨 ， 26 ℃ ~ 33 ℃ 
 　 　 紫外线 指数 
 　 　 中等 ， 戴好 遮阳帽 
 　 　 穿衣 指数 
 　 　 一级 ， 短裤 短袖 
 　 　 晨

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




来源:广州日报第01版,内容是十八 届 中央 巡视 回眸 
,被判断是抄袭新华社的消息
来源:长江日报第1版,内容是? ? ? ? 21 日 下午 ， 中考 英语 科目 考试 结束 , 考生 们 开心 地 走出 武汉市 第十七 中学 考点 ? ? ? 记者 彭年 ? 摄 
 ? ? ? ? 记者 陈晓彤 ? 杨幸 慈 ? 通讯员 张国伟 ? 黄 郁郁 
 ? ? ? ? 6 月 21 日 下午 ， 中考 最后 一门 科目 结束 ， 走出 考场 的 学生 们 一脸 轻松 。 长江日报 记者 采访 发现 ， 虽然 求学路 还 没 走 完 ， 在 这些 十五六岁 的 孩子 中 ， 不少 人 已经 对 高中 生活 甚至 职业 道路 有所 规划 。 
 ? ? ? ? 学业 ? 治好病 将来 做 物理学家 
 ? ? ? ? 黄子瑞 是 二桥 中学 学生 ， 他 用 “ 从容 ” 二字来 形容 自己 两天 来 的 考试 状态 。 “ 如果 给 自己 打分 的话 ， 心理 状态 10 分 ， 临场发挥 8 分 吧 。 ” 刚刚 走出 考场 ， 他 就 能 坦然 地 给 自己 的 中考 成绩 估分 ， “ 应该 有个 475 分 ， 高 一点 的话 490 分 ， 我 的 目标 是 武汉 三中 的 实验班 。 ” 
 ? ? ? ? 这个 学霸 考完 选择 用 看书 来 放松 自己 ， 暑假 里 黄子瑞 打算 提前 预习 高一 的 课程内容 。 “ 妈妈 想带 我 出去 旅行 ， 但 我 更 想 多 看点 书 。 ” 他 对 物理 尤其 感兴趣 ， “ 《 时间 简史 》 《 果壳 中 的 宇宙 》 《 大 设计 》 ， 买 了 一些 霍金的 书 ， 终于 有 时间 细细地 看 了 。 ” 
 ? ? ? ? 黄子瑞 在 体育 中考 前夕 查出 重疾 （ 肺动脉 高压 ） ， 家里 经济 原本 就 不 宽裕 ， 治病 又 是 一大笔 费用 ， 中考 过后 家人 准备 带 他 去 北京 继续 求医 。 虽然 黄子瑞 日益 消瘦 ， 平时 上楼 都 会 出现 呼吸困难 的 情况 ， 但 他 丝毫 没有 放弃 学习 的 想法 ， 很 坚强 也 很 阳光 ， 常 和 同学 说说笑笑 。 二桥 中学 九年 级 年级 组长 黄秀岚 说 ： “ 他 的 病 其实 很重 了 ， 目前 也 没有

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



一共8358条新闻被认定是抄袭新华社
判别出的抄袭新闻占总新闻的9.33%
随机森林模型结束


### Step10： 总结该过程，什么是数据思维？什么是机器学习思维？

答:
    数据思维包括多个维度,对于应用场景的运用理解,通过大型数据的统计分析和建模过程来理解数据给业务和场景带来的有效性和作用性等;
    机器学习思维可以简单理解为通过模拟人的思考方式,对输入的数据进行反复提取同样/不一样的经验,来对不同业务和场景进行预测性地
    自我思考和推动数据应用的过程.

### (Optional)使用第4课讲解的 edit distance，在涉嫌抄袭的文章中，找到其重复的文字与被修改过的文字。 

### Step11: 利用第8课讲述的新模型，进行操作，感受其中不同的参数、模型对性能的影响。(在上面代码中已经进    行了调试)